In [59]:
#pip3 install torch torchvision torchaudio

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math

In [70]:
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dimension, n_heads):
        super(MultiHeadAttention, self).__init__()
        assert model_dimension % n_heads == 0
        self.dimension = model_dimension // n_heads
        self.n_heads = n_heads

        # Linear layers for Q, K, V, and output
        self.q = nn.Linear(model_dimension, model_dimension)
        self.k = nn.Linear(model_dimension, model_dimension)
        self.v = nn.Linear(model_dimension, model_dimension)
        self.o = nn.Linear(model_dimension, model_dimension)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.dimension)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention_weights = torch.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, model_dim = x.size()
        x = x.view(batch_size, seq_length, self.n_heads, self.dimension)
        return x.permute(0, 2, 1, 3)  # (batch_size, n_heads, seq_length, dimension)

    def combine_heads(self, x):
        batch_size, n_heads, seq_length, dimension = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(batch_size, seq_length, n_heads * dimension)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.q(Q))
        K = self.split_heads(self.k(K))
        V = self.split_heads(self.v(V))
        attention = self.scaled_dot_product_attention(Q, K, V, mask)
        combined = self.combine_heads(attention)
        return self.o(combined)

In [71]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, model_dimension, ff_dimension):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(model_dimension, ff_dimension)
        self.fc2 = nn.Linear(ff_dimension, model_dimension)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [72]:
class PositionalEncoding(nn.Module):
    def __init__(self, model_dimension, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, model_dimension)
        position = torch.arange(0, max_seq_length).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, model_dimension, 2).float() * -(math.log(10000.0) / model_dimension))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [73]:
class EncoderLayer(nn.Module):
    def __init__(self, model_dimension, n_heads, ff_dimension, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(model_dimension, n_heads)
        self.feed_forward = PositionWiseFeedForward(model_dimension, ff_dimension)
        self.norm1 = nn.LayerNorm(model_dimension)
        self.norm2 = nn.LayerNorm(model_dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        return self.norm2(x + self.dropout(ff_output))

Now, we will be working on the decoder blocks

In [74]:
class DecoderLayer(nn.Module):
    def __init__(self, model_dimension, n_heads, ff_dimension, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(model_dimension, n_heads)
        self.cross_attn = MultiHeadAttention(model_dimension, n_heads)
        self.feed_forward = PositionWiseFeedForward(model_dimension, ff_dimension)
        self.norm1 = nn.LayerNorm(model_dimension)
        self.norm2 = nn.LayerNorm(model_dimension)
        self.norm3 = nn.LayerNorm(model_dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        cross_attn_output = self.cross_attn(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))
        ff_output = self.feed_forward(x)
        return self.norm3(x + self.dropout(ff_output))

Combining the Encoder and Decoder layers to create the complete Transformer network

In [75]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, model_dimension, n_heads, num_layers, ff_dimension, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, model_dimension)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, model_dimension)
        self.positional_encoding = PositionalEncoding(model_dimension, max_seq_length)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(model_dimension, n_heads, ff_dimension, dropout) for _ in range(num_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(model_dimension, n_heads, ff_dimension, dropout) for _ in range(num_layers)]
        )
        self.fc = nn.Linear(model_dimension, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
        seq_length = tgt.size(1)
        nopeak_mask = torch.triu(torch.ones(seq_length, seq_length), diagonal=1).bool().to(tgt.device)
        tgt_mask = tgt_mask & ~nopeak_mask.unsqueeze(0)
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)

        return self.fc(dec_output)

Training the Model

In [76]:
src_vocab_size = 5000
tgt_vocab_size = 5000
model_dimension = 512
n_heads = 8
num_layers = 6
ff_dimension = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, model_dimension, n_heads, num_layers, ff_dimension, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [77]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, model_dimension, n_heads, num_layers, ff_dimension, max_seq_length, dropout)

In [78]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.685275077819824
Epoch: 2, Loss: 8.559683799743652
Epoch: 3, Loss: 8.489509582519531
Epoch: 4, Loss: 8.427210807800293
Epoch: 5, Loss: 8.363287925720215
Epoch: 6, Loss: 8.290757179260254
Epoch: 7, Loss: 8.207722663879395
Epoch: 8, Loss: 8.120953559875488
Epoch: 9, Loss: 8.046219825744629
Epoch: 10, Loss: 7.957241535186768
Epoch: 11, Loss: 7.873353958129883
Epoch: 12, Loss: 7.792331218719482
Epoch: 13, Loss: 7.703634738922119
Epoch: 14, Loss: 7.618618965148926
Epoch: 15, Loss: 7.538391590118408
Epoch: 16, Loss: 7.4585723876953125
Epoch: 17, Loss: 7.377705097198486
Epoch: 18, Loss: 7.290804386138916
Epoch: 19, Loss: 7.211251735687256
Epoch: 20, Loss: 7.128533840179443
Epoch: 21, Loss: 7.050246715545654
Epoch: 22, Loss: 6.978647708892822
Epoch: 23, Loss: 6.901278018951416
Epoch: 24, Loss: 6.8246002197265625
Epoch: 25, Loss: 6.745556354522705
Epoch: 26, Loss: 6.677863121032715
Epoch: 27, Loss: 6.591556549072266
Epoch: 28, Loss: 6.528134822845459
Epoch: 29, Loss: 6.45175075